# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  -  #### 1️⃣. [infer NUCLEI ](./01_infer_nuclei.ipynb) 
  -  #### 2️⃣. [Infer SOMA](./02_infer_soma.ipynb) (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)
  -  #### 3️⃣. [Infer CYTOSOL](./03_infer_cytosol.ipynb)
  -  #### 4️⃣. [Infer LYSOSOMES](./04_infer_lysosome.ipynb)
  -  #### 5️⃣. [Infer MITOCHONDRIA](./05_infer_mitochondria.ipynb)
  -  #### 6️⃣. [Infer GOLGI complex](./06_golgi.ipynb)
  -  #### 7️⃣. [Infer PEROXISOMES](./07_peroxisome.ipynb)
  -  #### 8️⃣. [Infer ENDOPLASMIC RETICULUM ](./08_endoplasmic_reticulum.ipynb)
  -  #### 9️⃣. [Infer LB](./09_lipid_bodies.ipynb) 






## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
from scipy.ndimage import median_filter
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice, edge_preserving_smoothing_3d )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles import *

from infer_subc_2d.constants import TEST_IMG_N

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N


------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)


test_img_name = img_file_list[test_img_n]


In [3]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [8]:

ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 4,
    "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
    "nuc_ch" : 0,
    "optimal_Z": optimal_Z,
})


save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


In [ ]:
#optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
img_data.shape

## get the inferred nuclei object

(takes < 1 sec)

In [ ]:

###################
# INPUT NU
###################


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


In [ ]:

###################
# GET SOMA
###################

raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, out_p) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 

nuclei  = apply_mask(raw_nuclei, SO_object)
NU_object  = apply_mask(NU_object, SO_object)


Builde the segmentations in order




In [ ]:

###################
# INPUT LY
###################

raw_lyso =  img_2D[1].copy()
LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

In [ ]:

###################
# INPUT MT
###################
raw_mito    = img_2D[2].copy()

MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 


In [ ]:

###################
# INPUT GOLGI
###################
raw_golgi    = img_2D[3].copy()

GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 



In [ ]:

###################
# INPUT PEROXISOME
###################
raw_peroxi    = img_2D[4].copy()

PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

In [ ]:

###################
# INPUT ER
###################
raw_er  = img_2D[5].copy()

ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 


###################
# INPUT LIPID DRP
###################
raw_lipid   = img_2D[6].copy()

LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 


## View

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    raw_lyso,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    LY_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_mito,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    MT_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_golgi,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)

viewer.add_image(
    GL_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    raw_peroxi,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)

viewer.add_image(
    PO_object,
    scale=scale,
    blending='additive')


viewer.add_image(
    raw_er,
    scale=scale,
    colormap='red', 
    blending='additive'
)

viewer.add_image(
    ER_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    raw_lipid,
    scale=scale,
    colormap='red', 
    blending='additive'
)

viewer.add_image(
    LD_object,
    scale=scale,
    blending='additive')


In [ ]:
viewer.add_labels(
    label(ER_object),
    scale=scale,
    blending='additive')


In [ ]:

viewer.add_image(
    nuclei,
    scale=scale,
    colormap='blue', 
    blending='additive'
)

viewer.add_image(
    NU_object,
    scale=scale,
    blending='additive')



There may be a bug where the input images to the "infer_*" functions are modified in place and we might need to access them.  _MASKING_ seems to be the problem.  Also need to be clear about _when_ to apply the mask.

In [6]:

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 4,
    "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
    "nuc_ch" : 0,
})
ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0


In [ ]:

for i,target_file in enumerate(img_file_list):



    bioim_image = read_input_image(target_file)
    img_data = bioim_image.image
    raw_meta_data = bioim_image.raw_meta
    ome_types = []
    meta_dict = bioim_image.meta

    # get some top-level info about the RAW data
    channel_names = meta_dict['name']
    img = meta_dict['metadata']['aicsimage']
    scale = meta_dict['scale']
    channel_axis = meta_dict['channel_axis']

    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    # make sure we have removed Z
    if len(scale)>2:
        scale = scale[1:]

    img_2D = img_data[:,[optimal_Z],:,:].copy()

    raw_nuclei = img_2D[0].copy()
    NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 

    raw_soma = (4. * img_2D[1].copy() + 
                                1. * img_2D[5].copy() + 
                                1. * img_2D[7].copy() )

    SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, out_p) 
    CY_object =  infer_CYTOSOL(SO_object, NU_object) 


    nuclei  = apply_mask(raw_nuclei, SO_object)
    NU_object  = apply_mask(NU_object, SO_object)


    raw_lyso =  img_2D[1].copy()
    LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

    raw_mito    = img_2D[2].copy()
    MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 

    raw_golgi    = img_2D[3].copy()
    GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 

    raw_peroxi    = img_2D[4].copy()
    PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

    raw_er  = img_2D[5].copy()
    ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 

    raw_lipid   = img_2D[6].copy()
    LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 

    img_layers = [NU_object,
                            LY_object,
                            GL_object,
                            PO_object,
                            ER_object,
                            LD_object,
                            CY_object,
                            SO_object]
    layer_names = ["nuclei","lysosome","golgi","peroxisome","er","lipid_drop","cytosol_mask",'soma_mask']

    img_out = np.stack(img_layers, axis = 0)

    # add params to metadata

    out_path = data_root_path / "inferred_objects" 
    img_name = 'binarized_' + target_file.split("/")[-1].split(".")[0]

    out_file_n = export_ome_tiff(img_out, meta_dict, img_name, str(out_path)+"/", layer_names)
    print(f"saved file: {out_file_n}")




In [10]:


test_img = img_data.copy()
test_im2 = simple_intensity_normalization(test_img)

test_img.max(), test_im2.max(), img_data.max()


(65535, 1.0, 65535)

In [16]:


target_file = img_file_list[0]
##########
# IMPORT
##########

def  _import_czi_file(target_file):
    bioim_image = read_input_image(target_file)
    img_data = bioim_image.image
    raw_meta_data = bioim_image.raw_meta
    ome_types = []
    meta_dict = bioim_image.meta
    return (img_data, meta_dict)

img_data,meta_dict = _import_czi_file(target_file)
###########
# Choose Z
##########
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    # make sure we have removed Z

def _raw_soma_MCZ(img_in):
    """ define soma image
    """
    SOMA_W = (4.,1.,1.)
    SOMA_CH = (1,5,7) #LYSO, ER, RESIDUAL
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(SOMA_W,SOMA_CH):
        img_out += w*img_in[ch]
    return img_out


###########
# infer organelles
##########
def _infer_organelles(img_data, optimal_Z, default_params):
    NUC_ch = 1
    LYSO_ch = 1
    MITO_ch = 2
    GOLGI_ch = 3
    PEROXI_ch = 4
    ER_ch = 5
    LIPID_ch = 6

    # Stage 1:  nuclei, soma, cytosol
    img_2D = img_data[:,[optimal_Z],:,:].copy()

    raw_nuclei = img_2D[NUC_ch].copy()
    NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei, default_params) 

    raw_soma = _raw_soma_MCZ(img_2D)

    SO_object, SO_label, out_p =  infer_SOMA(raw_soma, NU_label, out_p) 
    CY_object =  infer_CYTOSOL(SO_object, NU_object) 


    #nuclei  = apply_mask(raw_nuclei, SO_object)
    NU_object  = apply_mask(NU_object, SO_object)


    # stage 2:  lysosomes, mitochondria, golgi, perosisomes, er, lipid bodies.
    raw_lyso =  img_2D[LYSO_ch].copy()
    LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

    raw_mito    = img_2D[MITO_ch].copy()
    MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 

    raw_golgi    = img_2D[GOLGI_ch].copy()
    GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 

    raw_peroxi    = img_2D[PEROXI_ch].copy()
    PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

    raw_er  = img_2D[ER_ch].copy()
    ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 

    raw_lipid   = img_2D[LIPID_ch].copy()
    LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 

    img_layers = [NU_object,
                            LY_object,
                            GL_object,
                            PO_object,
                            ER_object,
                            LD_object,
                            CY_object,
                            SO_object]
                            
    layer_names = ["nuclei","lysosome","golgi","peroxisome","er","lipid_drop","cytosol_mask",'soma_mask']

    img_out = np.stack(img_layers, axis = 0)
    return (img_out, layer_names, out_p)


inferred_organelles, layer_names, updated_params = _infer_organelles(img_data, optimal_Z, default_params)

##################
# export
##################

def _export_infer_organelles(img_out, layer_names, meta_dict):
       # get some top-level info about the RAW data
    channel_names = meta_dict['name']
    img = meta_dict['metadata']['aicsimage']
    scale = meta_dict['scale']
    channel_axis = meta_dict['channel_axis']
    if len(scale)>2:
        scale = scale[1:]



    # add params to metadata

    out_path = data_root_path / "inferred_objects" 
    img_name = 'binarized_' + target_file.split("/")[-1].split(".")[0]

    out_file_n = export_ome_tiff(img_out, meta_dict, img_name, str(out_path)+"/", layer_names)
    print(f"saved file: {out_file_n}")
    return out_file_n

out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict)



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['binarized_ZSTACK_PBTOhNGN2hiPSCs_BR3_N08_Unmixed']
saved file: /Users/ahenrie/Projects/Imaging/data/inferred_objects/binarized_ZSTACK_PBTOhNGN2hiPSCs_BR3_N08_Unmixed.ome.tiff


In [17]:
out_file_n

'/Users/ahenrie/Projects/Imaging/data/inferred_objects/binarized_ZSTACK_PBTOhNGN2hiPSCs_BR3_N08_Unmixed.ome.tiff'

In [12]:


target_file = img_file_list[0]
##########
# IMPORT
##########

def  _import_czi_file(target_file):
    bioim_image = read_input_image(target_file)
    img_data = bioim_image.image
    raw_meta_data = bioim_image.raw_meta
    ome_types = []
    meta_dict = bioim_image.meta
    return (img_data, meta_dict)

img_data,meta_dict = _import_czi_file(target_file)
###########
# Choose Z
##########
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    # make sure we have removed Z


def _raw_soma_MCZ(img_in):
    """ define soma image
    """
    SOMA_W = (4.,1.,1.)
    SOMA_CH = (1,5,7) #LYSO, ER, RESIDUAL
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(SOMA_W,SOMA_CH):
        img_out += w*img_in[ch]
    return img_out


NUC_ch = 0
LYSO_ch = 1
MITO_ch = 2
GOLGI_ch = 3
PEROXI_ch = 4
ER_ch = 5
LIPID_ch = 6

# Stage 1:  nuclei, soma, cytosol
img_2D = img_data[:,[optimal_Z],:,:].copy()

raw_nuclei = img_2D[NUC_ch].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei, default_params) 


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [13]:

raw_soma = _raw_soma_MCZ(img_2D)


In [15]:

SO_object, SO_label, out_p =  infer_SOMA(raw_soma, NU_label, out_p) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [10]:
raw_soma.shape

(8, 1, 768, 768)

In [20]:
in_img = raw_soma

out_p = out_p.copy()
struct_img = in_img.copy()
###################
# PRE_PROCESSING
###################

# TODO: replace params below with the input params
scaling_param = [0]
struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
out_p["intensity_norm_param"] = scaling_param

# make a copy for post-post processing
scaled_signal = struct_img.copy()

# Linear-ish processing
med_filter_size = 15
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
#struct_img = median_filter_slice_by_slice(struct_img, size=med_filter_size)
struct_img = median_filter(struct_img.squeeze(), size=med_filter_size)[np.newaxis,:,:]


intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [ ]:

out_p["median_filter_size"] = med_filter_size
gaussian_smoothing_sigma = 1.0
gaussian_smoothing_truncate_range = 3.0
struct_img = image_smoothing_gaussian_slice_by_slice(
    struct_img, sigma=gaussian_smoothing_sigma, truncate_range=gaussian_smoothing_truncate_range
)
out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma
out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

# non-Linear processing
log_img, d = log_transform(struct_img)
log_img = intensity_normalization(log_img, scaling_param=[0])

struct_img = intensity_normalization(scharr(log_img), scaling_param=[0]) + log_img

###################
# CORE_PROCESSING
###################
local_adjust = 0.5
low_level_min_size = 100
# "Masked Object Thresholding" - 3D
struct_obj, _bw_low_level = MO(
    struct_img,
    global_thresh_method="ave",
    object_minArea=low_level_min_size,
    extra_criteria=True,
    local_adjust=local_adjust,
    return_object=True,
    dilate=True,
)

out_p["local_adjust"] = local_adjust
out_p["low_level_min_size"] = low_level_min_size

###################
# POST_PROCESSING
###################

# 2D
hole_max = 80
struct_obj = hole_filling(struct_obj, hole_min=0.0, hole_max=hole_max**2, fill_2d=True)
out_p["hole_max"] = hole_max

small_object_max = 35
struct_obj = size_filter(
    struct_obj,  # wrapper to remove_small_objects which can do slice by slice
    min_size=small_object_max**3,
    method="slice_by_slice",
    connectivity=1,
)
out_p["small_object_max"] = small_object_max

labels_out = watershed(
    connectivity=np.ones((3, 3, 3), bool),
    image=1.0 - struct_img,
    markers=NU_labels,
    mask=np.logical_or(struct_obj, NU_labels > 0),
)
###################
# POST- POST_PROCESSING
###################
# keep the "SOMA" label which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [scaled_signal[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]

# now use all the NU labels which AREN't keep_label and add to mask and re-label
masked_composite_soma = struct_img.copy()
new_NU_mask = np.logical_and(NU_labels != 0, NU_labels != keep_label)

# "Masked Object Thresholding" - 3D
masked_composite_soma[new_NU_mask] = 0
struct_obj, _bw_low_level = MO(
    masked_composite_soma,
    global_thresh_method="ave",
    object_minArea=low_level_min_size,
    extra_criteria=True,
    local_adjust=local_adjust,
    return_object=True,
    dilate=True,
)

struct_obj = hole_filling(struct_obj, hole_min=0.0, hole_max=hole_max**2, fill_2d=True)
struct_obj = size_filter(
    struct_obj,  # wrapper to remove_small_objects which can do slice by slice
    min_size=small_object_max**3,
    method="slice_by_slice",
    connectivity=1,
)
masked_labels_out = watershed(
    connectivity=np.ones((3, 3, 3), bool),
    image=1.0 - struct_img,
    markers=NU_labels,
    mask=np.logical_or(struct_obj, NU_labels == keep_label),
)

retval = (struct_obj, masked_labels_out, out_p)


In [ ]:

CY_object =  infer_CYTOSOL(SO_object, NU_object) 



In [ ]:

#nuclei  = apply_mask(raw_nuclei, SO_object)
NU_object  = apply_mask(NU_object, SO_object)


# stage 2:  lysosomes, mitochondria, golgi, perosisomes, er, lipid bodies.
raw_lyso =  img_2D[LYSO_ch].copy()
LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

raw_mito    = img_2D[MITO_ch].copy()
MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 

raw_golgi    = img_2D[GOLGI_ch].copy()
GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 

raw_peroxi    = img_2D[PEROXI_ch].copy()
PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

raw_er  = img_2D[ER_ch].copy()
ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 

raw_lipid   = img_2D[LIPID_ch].copy()
LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 

img_layers = [NU_object,
                        LY_object,
                        GL_object,
                        PO_object,
                        ER_object,
                        LD_object,
                        CY_object,
                        SO_object]
                        
layer_names = ["nuclei","lysosome","golgi","peroxisome","er","lipid_drop","cytosol_mask",'soma_mask']

img_out = np.stack(img_layers, axis = 0)

In [ ]:
target_file

In [ ]:
img_out.shape, len(layer_names), data_path

In [ ]:
out_path = data_root_path / "inferred_objects" 
img_name = 'binarized_' + target_file.split("/")[-1].split(".")[0]

out_file_n = export_ome_tiff(img_out, meta_dict, img_name, str(out_path)+"/", layer_names)




In [ ]:
from aicsimageio.writers import OmeTiffWriter
data_in = img_out
channel_names = [layer_names]
image_names = [img_name]
print(image_names)
# chan_names = meta_in['metadata']['aicsimage'].channel_names
dimension_order = ["CZYX"]

num_images = len(  [data_in.shape])
if data_in.dtype == "bool":
    data_in = data_in.astype(np.uint8)
    data_in[data_in > 0] = 255

physical_pixel_sizes = [meta_dict["metadata"]["aicsimage"].physical_pixel_sizes]
out_ome = OmeTiffWriter.build_ome(
        [data_in.shape],
        [data_in.dtype],
        channel_names=channel_names,  # type: ignore
        image_name=image_names,
        physical_pixel_sizes=physical_pixel_sizes,
        dimension_order=dimension_order,
    )

In [ ]:
out_ome.images.append()

In [ ]:
raw_golgi2    = img_2D[3].copy()

viewer.add_image(
    raw_golgi2,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)


In [ ]:
CY_object.sum()

In [ ]:
viewer.add_image(img_out,
channel_axis = 0,
name = layer_names)

In [ ]:
viewer.add_image(img_data,
channel_axis = 0)

In [ ]:
napari.view_image(img_data)